# SparkSQL

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
spark = SparkSession.builder.appName("SparkSQL-basic").getOrCreate()

## Membuat Data

In [4]:
students_records = sc.parallelize([Row(roll_no=1, name='Intan', passed=True, marks={'Math':89,'Phycsics':87,'Chemistry':81},
                                       sports=['chess','foot-ball'], Dob=datetime(2012,5,1,13,1,5)),
                                   Row(roll_no=2, name='Smith', passed=False, marks={'Math':29,'Phycsics':31,'Chemistry':36},
                                       sports=['volley-ball','table-tennis'], Dob=datetime(2012,5,12,14,2,5))])

In [5]:
student_records_df = students_records.toDF()
student_records_df.show()

+-------+-----+------+--------------------+--------------------+-------------------+
|roll_no| name|passed|               marks|              sports|                Dob|
+-------+-----+------+--------------------+--------------------+-------------------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+------+--------------------+--------------------+-------------------+



In [6]:
student_records_df.show(truncate=False)

+-------+-----+------+---------------------------------------------+---------------------------+-------------------+
|roll_no|name |passed|marks                                        |sports                     |Dob                |
+-------+-----+------+---------------------------------------------+---------------------------+-------------------+
|1      |Intan|true  |{Phycsics -> 87, Chemistry -> 81, Math -> 89}|[chess, foot-ball]         |2012-05-01 13:01:05|
|2      |Smith|false |{Phycsics -> 31, Chemistry -> 36, Math -> 29}|[volley-ball, table-tennis]|2012-05-12 14:02:05|
+-------+-----+------+---------------------------------------------+---------------------------+-------------------+



## Membuat View

In [7]:
student_records_df.createOrReplaceTempView('records')

In [8]:
spark.sql("SELECT * FROM records").show()

+-------+-----+------+--------------------+--------------------+-------------------+
|roll_no| name|passed|               marks|              sports|                Dob|
+-------+-----+------+--------------------+--------------------+-------------------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+------+--------------------+--------------------+-------------------+



In [9]:
spark.sql("SELECT name FROM records").show()

+-----+
| name|
+-----+
|Intan|
|Smith|
+-----+



In [10]:
type(1)

int

In [11]:
results = spark.sql("SELECT * FROM records")
type(results)

pyspark.sql.dataframe.DataFrame

## Filter Data


In [14]:
spark.sql('SELECT roll_no, marks["Phycsics"], sports[1] FROM records').show()

+-------+---------------+------------+
|roll_no|marks[Phycsics]|   sports[1]|
+-------+---------------+------------+
|      1|             87|   foot-ball|
|      2|             31|table-tennis|
+-------+---------------+------------+



## Where Clause

In [15]:
spark.sql('SELECT * FROM records WHERE passed=True').show()

+-------+-----+------+--------------------+------------------+-------------------+
|roll_no| name|passed|               marks|            sports|                Dob|
+-------+-----+------+--------------------+------------------+-------------------+
|      1|Intan|  true|{Phycsics -> 87, ...|[chess, foot-ball]|2012-05-01 13:01:05|
+-------+-----+------+--------------------+------------------+-------------------+



In [16]:
spark.sql('SELECT * FROM records WHERE marks["Chemistry"] < 40').show()

+-------+-----+------+--------------------+--------------------+-------------------+
|roll_no| name|passed|               marks|              sports|                Dob|
+-------+-----+------+--------------------+--------------------+-------------------+
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+------+--------------------+--------------------+-------------------+



## Global View

In [17]:
student_records_df.createGlobalTempView('global_records')

In [18]:
spark.sql('SELECT * FROM global_temp.global_records').show()

+-------+-----+------+--------------------+--------------------+-------------------+
|roll_no| name|passed|               marks|              sports|                Dob|
+-------+-----+------+--------------------+--------------------+-------------------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+------+--------------------+--------------------+-------------------+



## Hapus Kolom

In [19]:
student_records_df.columns

['roll_no', 'name', 'passed', 'marks', 'sports', 'Dob']

In [25]:
student_records_df = student_records_df.drop('passed')
student_records_df.show()

+-------+-----+--------------------+--------------------+-------------------+
|roll_no| name|               marks|              sports|                Dob|
+-------+-----+--------------------+--------------------+-------------------+
|      1|Intan|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+--------------------+--------------------+-------------------+



In [26]:
student_records_df.createOrReplaceTempView('records_new')
spark.sql('SELECT * FROM records_new').show()

+-------+-----+--------------------+--------------------+-------------------+
|roll_no| name|               marks|              sports|                Dob|
+-------+-----+--------------------+--------------------+-------------------+
|      1|Intan|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+--------------------+--------------------+-------------------+



In [27]:
spark.sql('SELECT * FROM records').show()

+-------+-----+------+--------------------+--------------------+-------------------+
|roll_no| name|passed|               marks|              sports|                Dob|
+-------+-----+------+--------------------+--------------------+-------------------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|
+-------+-----+------+--------------------+--------------------+-------------------+



## Agregat = Kolom Baru

In [28]:
student_records_df.show(truncate=False)

+-------+-----+---------------------------------------------+---------------------------+-------------------+
|roll_no|name |marks                                        |sports                     |Dob                |
+-------+-----+---------------------------------------------+---------------------------+-------------------+
|1      |Intan|{Phycsics -> 87, Chemistry -> 81, Math -> 89}|[chess, foot-ball]         |2012-05-01 13:01:05|
|2      |Smith|{Phycsics -> 31, Chemistry -> 36, Math -> 29}|[volley-ball, table-tennis]|2012-05-12 14:02:05|
+-------+-----+---------------------------------------------+---------------------------+-------------------+



In [34]:
spark.sql('SELECT round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records').show()

+---------+
|avg_marks|
+---------+
|     86.0|
|     32.0|
+---------+



In [37]:
student_records_df = spark.sql('SELECT *, round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records')
student_records_df.show()

+-------+-----+------+--------------------+--------------------+-------------------+---------+
|roll_no| name|passed|               marks|              sports|                Dob|avg_marks|
+-------+-----+------+--------------------+--------------------+-------------------+---------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|     86.0|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|     32.0|
+-------+-----+------+--------------------+--------------------+-------------------+---------+



## Update View

In [38]:
student_records_df.createOrReplaceTempView('record_new')

In [39]:
student_records_df = student_records_df.withColumn('status', (when(col('avg_marks')>=40, 'passed')).otherwise('failed'))
student_records_df.show()

+-------+-----+------+--------------------+--------------------+-------------------+---------+------+
|roll_no| name|passed|               marks|              sports|                Dob|avg_marks|status|
+-------+-----+------+--------------------+--------------------+-------------------+---------+------+
|      1|Intan|  true|{Phycsics -> 87, ...|  [chess, foot-ball]|2012-05-01 13:01:05|     86.0|passed|
|      2|Smith| false|{Phycsics -> 31, ...|[volley-ball, tab...|2012-05-12 14:02:05|     32.0|failed|
+-------+-----+------+--------------------+--------------------+-------------------+---------+------+



## Data Baru

In [40]:
employeeData = (('John','HR','NY',90000,34,10000),('Neha','HR','NY',86000,28,20000),
                ('Robert','Sales','CA',81000,56,22000),('Maria','Sales','CA',99000,45,15000),
                ('Paul','IT','NY',98000,38,14000),('Jen','IT','CA',90000,34,20000),
                ('Raj','IT','CA',93000,28,28000),('Pooja','IT','CA',95000,31,19000))
columns = ('employee_name','department','state','salary','age','bonus')
employeeDf = spark.createDataFrame(employeeData, columns)
employeeDf.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|         John|        HR|   NY| 90000| 34|10000|
|         Neha|        HR|   NY| 86000| 28|20000|
|       Robert|     Sales|   CA| 81000| 56|22000|
|        Maria|     Sales|   CA| 99000| 45|15000|
|         Paul|        IT|   NY| 98000| 38|14000|
|          Jen|        IT|   CA| 90000| 34|20000|
|          Raj|        IT|   CA| 93000| 28|28000|
|        Pooja|        IT|   CA| 95000| 31|19000|
+-------------+----------+-----+------+---+-----+



## GroupBy

In [41]:
employeeDf.groupBy(col('department')).agg(sum(col('salary'))).show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|        HR|     176000|
|     Sales|     180000|
|        IT|     376000|
+----------+-----------+



In [42]:
employeeDf.groupBy(col('department')).agg(sum(col('salary'))).alias('total_sal').orderBy('total_sal').show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `total_sal` cannot be resolved. Did you mean one of the following? [`department`, `sum(salary)`].;
'Sort ['total_sal ASC NULLS FIRST], true
+- SubqueryAlias total_sal
   +- Aggregate [department#571], [department#571, sum(salary#573L) AS sum(salary)#636L]
      +- LogicalRDD [employee_name#570, department#571, state#572, salary#573L, age#574L, bonus#575L], false


In [ ]:
employeeDf.groupBy(col('department')).agg(sum(col('salary'))).alias('total_sal').orderBy(col('total_sal').desc()).show()

In [ ]:
employeeDf.groupBy(col('department'),col('state')).agg(sum(col('bonus'))).show()

In [ ]:
employeeDf.groupBy(col('department')).agg(avg(col('salary')).alias('avg_salary'),max(col('bonus')).alias('max_bonus')).show()